### 6-1 什么是梯度下降法

个人的总结：   
线性回归：简单线性回归、多元线性回归，其中假设函数是线性的。
对于简单线性回归： 
    * 对损失函数求导得到参数：最小二乘法。
    * 梯度下降法，逐渐找到使得损失函数最小的点对应的参数。
对于多元线性回归： 
    * 正规方程法
    * 设定合适的损失函数，各个方向趋近使得损失函数最小的点所对应的参数。

* 不是一个机器学习算法
* 是一种基于搜索的最优化的方法（即如何得到损失最小的损失函数）
* 作用：最小化一个损失函数（代价函数），我们上一节的方法是直接对损失函数求导，这里是一步一步到达最低点
* 梯度上升法：最大化一个效用函数

 **η 学习率**
   * 取值决定获得最优解的速度
   * 取值不合适时甚至得不到最优解
   * 是梯度下降法的一个超参数

要注意，在高维平面上，可能有多个极小值，在这种方式下，用梯度下降法找到的局部最优解不一定是全局最优解。\
解决办法：多次运行，随机化初始点
* 梯度下降的初始点也是一个超参数
* 在本章中在线性回归中使用梯度下降法，损失函数有唯一的最优解

### 梯度下降法的模拟

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plot_x = np.linspace(-1,6,141)
plot_x

In [ ]:
plot_y = (plot_x-2.5)**2-1

即代价函数 H(θ) = （θ-2.5）^2 -1 , 比如简单线性回归，假设函数y = a * x + b ， 固定一个截距b，得到的代价函数可能是这种形式（二维）

In [ ]:
plt.plot(plot_x,plot_y)
plt.show()

对上述图形的某个横轴求导

In [ ]:
def dJ(theta):
    return 2*(theta-2.5)

求损失函数

In [ ]:
def J(theta):
    return (theta-2.5)**2-1

In [ ]:
eta = 0.1 # 学习率
theta = 0
epsilon = 1e-8
while True:
    gradient = dJ(theta) # 计算梯度（所谓的斜率，单位上升/下降）
    last_theta = theta
    theta = theta - gradient * eta # 向梯度的相反方向移动，幅度是eta，这一步计算theta移动的距离（向什么方向、步长）
    # 到什么程度结束循环？到导数为0的点，这里设定极小值
    
    if(abs(J(theta)-J(last_theta)) < epsilon):
        break

print(theta)
print(J(theta))

查看eta如何决定了移动的方向

In [ ]:
eta = 0.1 
theta = 0
epsilon = 1e-8
theta_history = [theta] # 添加一个列表，记录横轴的theta的值


while True:
    gradient = dJ(theta) 
    last_theta = theta
    theta = theta - gradient * eta 
    theta_history.append(theta)
    
    if(abs(J(theta)-J(last_theta)) < epsilon):
        break
plt.plot(plot_x,plot_y,'b',)
plt.plot(theta_history,J(np.array(theta_history)),color='r',marker='+')
#plt.plot(theta_history,J(np.array(theta_history)))
plt.show()

封装上述代码，查看不同的eta对于梯度下降的影响

In [ ]:
def gradient_descent(initial_theta,eta,epsilon=1e-8):
    theta = initial_theta
    theta_history.append(theta)
    
    while True:
        gradient = dJ(theta) 
        last_theta = theta
        theta = theta - gradient * eta 
        theta_history.append(theta)
        if(abs(J(theta)-J(last_theta)) < epsilon):
            break
            
def plot_theta_history():
    plt.plot(plot_x,plot_y)
    plt.plot(np.array(theta_history),J(np.array(theta_history)),color='r',marker='+')
    plt.show()

In [ ]:
theta_history = []
gradient_descent(0.,0.01)
plot_theta_history()

In [ ]:
theta_history = []
gradient_descent(0.,0.001)
plot_theta_history()

In [ ]:
len(theta_history)

In [ ]:
theta_history = []
gradient_descent(0.,0.8)
plot_theta_history()

eta过大的时候

In [ ]:
# theta_history = []
# gradient_descent(0.,1.1)
# plot_theta_history()

需要重写代价函数/损失函数

In [ ]:
def J(theta):
    try:
        return (theta-2.5)**2-1.
    except:
        return float('inf')

In [ ]:
def gradient_descent(initial_theta,eta,n_iters = 1e4, epsilon=1e-8): # 由于可能不会逼近极值点，所以设置最大循环10000，防止进入死循环
    theta = initial_theta
    theta_history.append(theta)
    i_iter = 0 # 设置循环次数
    
    while i_iter < n_iters:
        gradient = dJ(theta) 
        last_theta = theta
        theta = theta - gradient * eta 
        theta_history.append(theta)
        if(abs(J(theta)-J(last_theta)) < epsilon):
            break
        i_iter += 1

In [ ]:
theta_history = []
gradient_descent(0.,1.1)

In [ ]:
len(theta_history)

In [ ]:
theta_history = []
gradient_descent(0.,1.1,10)
plot_theta_history() # 损失函数越来越大

### 6.3 多元线性回归中的梯度下降法

上述将损失（代价）函数看成一个二次曲线，来模拟梯度下降的过程；这里引入多元线性回归的梯度下降法。 
注意之前学过，如果假设函数是y = a + bx，其代价函数就像是一个碗。

* 假设函数是$\widehat{y} = \theta_0 + {\theta_1}{X_1^{(i)}}+{\theta_2}{X_2^{(i)}}+{\theta_3}{X_3^{(i)}}+ ... +{\theta_n}{X_n^{(i)}} $ 

* 使得代价函数$J(\theta)=\sum_{i=1}^m(y^{(i)}-\widehat{y}^{(y)})^{2}$取得最小值
* 下代入上， 即使得$$J(\theta)=\sum_{i=1}^m(y^{(i)}-\theta_0 - {\theta_1}{X_1^{(i)}}-{\theta_2}{X_2^{(i)}} ... -{\theta_n}{X_n^{(i)}})^{2}$$取得最小值
* 对代价函数的各个分量求偏导J
 * $\frac{\partial{J}}{\partial{\theta_j}} = \sum_{i=1}^m 2(y^{(i)}-\theta_0 - {\theta_1}{X_1^{(i)}}-{\theta_2}{X_2^{(i)}} ... -{\theta_n}{X_n^{(i)}})\cdot{(-X_j^{(i)})} = \sum_{i=1}^m 2(y^{(i)}-X_b^{(i)}\theta)\cdot(-X_j^{(i)})$ 

* 为了使得梯度 $\frac{\partial{J}}{\partial{\theta_j}}$和样本容量大小本身没有关系，我们在上个式子乘以$\frac{1}{m}$，有的教材是（$\frac{2}{m}$），将代价函数改为$J(\theta)= \frac{1}{m}\sum_{i=1}^m(y^{(i)}-\widehat{y}^{(y)})^{2}$，此时梯度为
$\frac{\partial{J}}{\partial{\theta_j}} = \sum_{i=1}^m \frac{2}{m}\cdot (y^{(i)}-X_b^{(i)}\theta)\cdot(-X_j^{(i)})$ 

* 其实这里的代价函数就是均方误差 $\frac{\partial{J}}{\partial{\theta_j}} = MSE(y,\hat{y})$

### 6-4 在线性回归模型中使用梯度下降法

**制造一个虚假的例子，用一元来模拟多元的情况便于可视化**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(666)
x = 2 * np.random.random(size = 100)
x
y = x * 3. + 4. + np.random.normal(size=100) # 加上符合标准正太分布的噪音
y

In [ ]:
X = x.reshape(-1,1) # 第二个维度为1 ,100行1列
X.shape

In [ ]:
y.shape

In [ ]:
plt.scatter(X,y,c='r',s=1)
plt.xlabel("x_value(single feature)")
plt.ylabel("y_value")
plt.show()

**复习：之前到这一步，我们是怎么做的？** 
* 假设函数$\hat{y}= \theta_0 + \theta_1\cdot x$
* 代价函数 $\sum_{i=1}^{m}(y^{(i)}-\hat{y}^{(i)})^{2}$,使其最小，直接求导，使导数==0

**现在我们用梯度下降法，首先设置假设函数**

注意是矩阵运算： 
* theta是(n+1)*1矩阵，第一个数字是截距，剩下的是斜率参数
* X_b是m个样本n个特征组成的m*(n+1)的矩阵,第一列加上一列1
* y是样本的值（监督学习），是m*1的矩阵
* 返回的是代价函数的一个值

In [ ]:
def J(theta,X_b,y):
    try:
        return np.sum((y-X_b.dot(theta))**2) / len(X_b)
    except:
        return float("inf")

In [ ]:
def dJ(theta,X_b,y):
    res = np.empty(len(theta)) # n个维度，将求出n个导数
    res[0] = np.sum(X_b.dot(theta)-y)
    for i in range(1,len(theta)):
        res[i] = (X_b.dot(theta)-y).dot(X_b[:,i])
    return res * 2 / len(X_b)
    

In [ ]:
def gradient_descent(X_b,y,initial_theta,eta,n_iters = 1e4, epsilon=1e-8):
    theta = initial_theta
    i_iter = 0 # 设置循环次数
    
    while i_iter < n_iters:
        gradient = dJ(theta,X_b,y) 
        last_theta = theta
        theta = theta - gradient * eta 
        theta_history.append(theta)
        if(abs(J(theta,X_b,y)-J(last_theta,X_b,y)) < epsilon):
            break
        i_iter += 1
    return theta

In [ ]:
X_b = np.hstack((np.ones((len(x),1)),x.reshape(-1,1)))
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01

theta = gradient_descent(X_b,y,initial_theta,eta)
theta

在Linear Regression包中实现自己写的代码

In [ ]:
from playML.LinearRegression import LinearRegression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit_gd(X,y)

In [ ]:
lin_reg.coef_

In [ ]:
lin_reg.interception_

### 6-5 线性回归梯度下降法的向量化

**向量计算**  
注意在numpy中一维向量不区分横竖，会自动计算一维向量的横竖，（这里我们把一维计算结果默认为列），这里为了严谨区分了横竖，其实无论写不写$^T$，结果都一样
$ DJ = \frac{DJ}{D\theta} = \vec{\theta} = \frac{2}{m}\cdot (X_b\theta-y)^T\cdot X_b$，  
或者 $ DJ = \frac{DJ}{D\theta} = \vec{\theta} = \frac{2}{m}\cdot X_b^T \cdot (X_b\theta-y)$ 变成横向量


In [ ]:
import numpy as np

点乘运算会自动判别一维向量是横向量还是纵向量

In [ ]:
A = np.array(range(16)).reshape(4,-1)
A

In [ ]:
b = np.array([1,2,3,4])

In [ ]:
A.T.dot(b)

In [ ]:
b.dot(A)

In [ ]:
from sklearn import datasets 

In [ ]:
boston = datasets.load_boston()
boston.keys()
boston.data.shape
X = boston.data
y = boston.target

X = X[y<50.0]
y = y[y<50.0]

In [ ]:
from playML.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,seed = 666)

In [ ]:
from playML.LinearRegression import LinearRegression

In [ ]:
lin_regl = LinearRegression()
%time lin_regl.fit_normal(X_train,y_train)
lin_regl.score(X_test,y_test)

In [ ]:
lin_reg2 = LinearRegression()
%time lin_reg2.fit_gd(X_train,y_train)

In [ ]:
lin_reg2.coef_

原因：真实的数据集：每一个特征的数据规模完全不同，eta即使设置得很小，但是步长还是太大，结果不收敛。

In [ ]:
lin_reg2 = LinearRegression()
%time lin_reg2.fit_gd(X_train,y_train,eta=0.000001)
lin_reg2.score(X_test,y_test)

为什么成功率这么低？可能是eta太小了，在有限的次数下难以收敛

In [ ]:
lin_reg2 = LinearRegression()
%time lin_reg2.fit_gd(X_train,y_train,eta=0.000001,n_iters=1e6)
lin_reg2.score(X_test,y_test)

解决方式：数据归一化

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
standardScaler = StandardScaler()
standardScaler.fit(X_train)

In [ ]:
X_train_standard = standardScaler.transform(X_train)
X_train_standard.shape

In [ ]:
lin_reg3 = LinearRegression()
lin_reg3.fit_gd(X_train_standard,y_train)

In [ ]:
X_test_standard = standardScaler.transform(X_test)

In [ ]:
lin_reg3._theta

In [ ]:
lin_reg3.score(X_test_standard,y_test)

梯度下降法相对于正规方程法的优势在于：特征数越多，梯度下降法相对耗时短。（矩阵运算慢）

### 6-6 随机梯度下降法 Stochastic  Gradient Descent

**考虑一种情况，给定的样本数量极其庞大**，为了方便，在求偏导的时候，只对其中的一个样本做计算，即 $$ = \frac{2}{m}\left[ \begin{matrix} \sum_{i=1}^{m} (X_b^{(i)}\theta-y^{(i)})\cdot X_0^{(i)} \\ \sum_{i=1}^{m} (X_b^{(i)}\theta-y^{(i)})\cdot X_1^{(i)} \\ \sum_{i=1}^{m} (X_b^{(i)}\theta-y^{(i)})\cdot X_2^{(i)}\\ \sum_{i=1}^{m} (X_b^{(i)}\theta-y^{(i)}) \cdot X_3^{(i)} \\ \cdots \\ \sum_{i=1}^{m} (X_b^{(i)}\theta-y^{(i)})\cdot X_n^{(i)}\end{matrix} \right] $$  $$=2 \cdot \left[ \begin{matrix} \ (X_b^{(i)}\theta-y^{(i)})\cdot X_0^{(i)} \\ (X_b^{(i)}\theta-y^{(i)})\cdot X_1^{(i)} \\  (X_b^{(i)}\theta-y^{(i)})\cdot X_2^{(i)}\\  (X_b^{(i)}\theta-y^{(i)})\cdot X_3^{(i)} \\ \cdots \\  (X_b^{(i)}\theta-y^{(i)}).\cdot X_n^{(i)}\end{matrix} \right]$$$$=2\cdot(X_b^{(i)})^T\cdot(X_b^{(i)}\theta-y^{(i)})$$

按照之前的梯度下降，会沿着损失函数最小值的方向向下移动，但是那样做在样本值很大的情况下，会导致每个方向都要计算，计算量很大；这样选择一个样本计算，可以下降的道路比较曲折，但是方向是正确的。

**关于学习率要注意以下几点** 
* 学习率的取值非常重要。由于随机过程不好，容易跳出最小值所在的位置，所以实际上学习率是逐渐递减的。设计一个关于循环次数的函数。
* $\eta = \frac{1}{i_{-}iters}$
* 当循环次数小的时候，$\eta$下降的速度实在太快了，所以给分母加上一个数$\eta = \frac{1}{i_{-}iters+b}$，b可以取50
* 分子的位置，取一个常数，使之更灵活，比如a
* 这里的a,b就是超参数
* 这是搜索领域的模拟退火的思想

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
m = 100000
x = np.random.normal(size=m)
X = x.reshape(-1,1)
y = 4. * x + 3. + np.random.normal(0,3,size=m)

In [ ]:
def J(theta,X_b,y):
    try:
        return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
    except:
        return float('inf')

def dJ(theta,X_b,y):
    return X_b.T.dot(X_b.dot(theta)-y) * 2. / len(y)

def gradient_descent(X_b,y,initial_theta,eta,n_iters=1e4,epsilon=1e-8):
    theta = initial_theta
    cur_iter = 0
    while cur_iter < n_iters:
        gradient = dJ(theta,X_b,y)
        last_theta = theta
        theta = theta - eta * gradient
        if abs(J(theta,X_b,y)-J(last_theta,X_b,y))<epsilon:
            break
        cur_iter += 1
    return theta

In [ ]:
%%time
X_b = np.hstack([np.ones((len(X),1)),X])
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
theta = gradient_descent(X_b,y,initial_theta,eta)

In [ ]:
theta

**使用随机梯度下降法**

In [ ]:
def J(theta,X_b,y):
    try:
        return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
    except:
        return float('inf')
    
# def dJ(theta,X_b,y):
#     return X_b.T.dot(X_b.dot(theta)-y) * 2. / len(y)

#求梯度的方向
def dJ_sgd(theta,X_b_i,y_i):
    return X_b_i.T.dot(X_b_i.dot(theta)-y_i) * 2. 

def sgd(X_b,y,initial_theta,n_iters):
    t0 = 5
    t1 = 50
    
    def learning_rate(t):
        return t0 / (t + t1)
    theta = initial_theta
    
    for cur_iter in range(n_iters):
        rand_i = np.random.randint(len(X_b))
        gradient = dJ_sgd(theta,X_b[rand_i],y[rand_i])
        theta = theta - learning_rate(cur_iter)*gradient
    return theta



In [ ]:
%%time
X_b = np.hstack([np.ones((len(X),1)),X])
initial_theta = np.zeros(X_b.shape[1])
theta = sgd(X_b,y,initial_theta,n_iters = len(X_b)//3)

In [ ]:
theta

### 6-7 scikit-learn中的随机梯度下降法

将随机梯度下降法封装在自己的函数中

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
m = 100000

x = np.random.normal(size=m)
X = x.reshape(-1,1)
y = 4. *x + 3. + np.random.normal(0,3,size=m)

In [4]:
from playML.LinearRegression import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit_sgd(X,y,n_iters=2)

LinearRegression()

In [5]:
lin_reg.coef_

array([3.97909507])

In [6]:
lin_reg.interception_

2.9894919785949567

使用真实的数据，使用自己的sgd

In [7]:
from sklearn import datasets
boston = datasets.load_boston()
boston.keys()
boston.data.shape
X = boston.data
y = boston.target

X = X[y<50.0]
y = y[y<50.0]

In [9]:
from playML.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,seed = 666)

In [10]:
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
standardScaler.fit(X_train)
x_train_standard = standardScaler.transform(X_train)
x_test_standard = standardScaler.transform(X_test)

In [11]:
from playML.LinearRegression import LinearRegression

lin_reg = LinearRegression()
%time lin_reg.fit_sgd(x_train_standard,y_train,n_iters=2)
lin_reg.score(x_test_standard,y_test)

Wall time: 11 ms


0.7857275413602651

当观察的样本整体的数量增加，结果也更加准确了。

In [12]:
from playML.LinearRegression import LinearRegression

lin_reg = LinearRegression()
%time lin_reg.fit_sgd(x_train_standard,y_train,n_iters=50)
lin_reg.score(x_test_standard,y_test)

Wall time: 148 ms


0.808560757055621

在scikit-learn中的SGD

In [14]:
from sklearn.linear_model import SGDRegressor

In [15]:
sgd_reg = SGDRegressor()

In [16]:
%time sgd_reg.fit(x_train_standard,y_train)

Wall time: 30 ms


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [17]:
sgd_reg.score(x_test_standard,y_test)

0.8123287969388414

在sgd_reg = SGDRegressor()可以传入参数：样本整体浏览多少次，默认是5；如sgd_reg = SGDRegressor(n_iter=5)

### 6-8 如何确定梯度下降法的准确性 调试梯度下降法

### 6-9 有关梯度下降法的更多的深入讨论